In [61]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath('../'))
from task_vectors import TaskVector
from evaluation import eval_single_dataset

In [63]:
pretrained_checkpoint = f'artificial_checkpoints/mlp_model.pth'
finetuned_checkpoint2 = f'artificial_checkpoints/mlp_model_dataset2.pth'
finetuned_checkpoint3 = f'artificial_checkpoints/mlp_model_dataset3.pth'
dataset2 = 'dataset2'
dataset3 = 'dataset3'
device = 'cpu'

In [64]:
task_vector1 = TaskVector(
            pretrained_checkpoint=pretrained_checkpoint,
            finetuned_checkpoint=finetuned_checkpoint2,
            operation='subtract',
        )

In [65]:
task_vector2 = TaskVector(
            pretrained_checkpoint=pretrained_checkpoint,
            finetuned_checkpoint=finetuned_checkpoint3,
            operation='subtract',
        )

In [66]:
sum_task_vector = task_vector2 + task_vector1

In [67]:
model_backbone = sum_task_vector.apply_to(pretrained_checkpoint, 0.5)

In [68]:
# generate a range vector between 0.1 and 0.7

vector = np.arange(0.1, 0.95, 0.05)

In [69]:
results = []
for i in vector:
    model_backbone = sum_task_vector.apply_to(pretrained_checkpoint, i)
    accA = eval_single_dataset(model_backbone, dataset2, device)
    accB = eval_single_dataset(model_backbone, dataset3, device)
    results.append((i, accA, accB))


In [70]:
results

[(0.1, 0.19041666388511658, 0.04874999821186066),
 (0.15000000000000002, 0.19958333671092987, 0.05000000074505806),
 (0.20000000000000004, 0.19958333671092987, 0.05000000074505806),
 (0.25000000000000006, 0.19958333671092987, 0.05000000074505806),
 (0.30000000000000004, 0.19958333671092987, 0.05000000074505806),
 (0.3500000000000001, 0.19958333671092987, 0.05000000074505806),
 (0.40000000000000013, 0.19958333671092987, 0.05000000074505806),
 (0.4500000000000001, 0.19958333671092987, 0.05000000074505806),
 (0.5000000000000001, 0.19958333671092987, 0.05000000074505806),
 (0.5500000000000002, 0.19958333671092987, 0.05000000074505806),
 (0.6000000000000002, 0.20291666686534882, 0.05000000074505806),
 (0.6500000000000001, 0.3645833432674408, 0.05000000074505806),
 (0.7000000000000002, 0.41083332896232605, 0.05000000074505806),
 (0.7500000000000002, 0.36541667580604553, 0.05062500014901161),
 (0.8000000000000003, 0.3166666626930237, 0.04937500134110451),
 (0.8500000000000003, 0.2829166650772

## Interpolation test

In [71]:
results = []
for interpolation_value in np.arange(0.1, 0.95, 0.05):
    sum_interpolate = task_vector2.sub_interpolate(task_vector1, interpolation_value)
    for alpha in np.arange(0.1, 0.95, 0.05):
        model_backbone = sum_interpolate.apply_to(pretrained_checkpoint, alpha)
        accA = eval_single_dataset(model_backbone, dataset2, device)
        accB = eval_single_dataset(model_backbone, dataset3, device)
        results.append((interpolation_value, alpha, accA, accB))
# Fine-tuning parameters

In [72]:
results_df = pd.DataFrame(results, columns=['interpolation_value', 'alpha', 'accA', 'accB'])

In [73]:
results_df

,interpolation_value,alpha,accA,accB
0,0.1,0.10,0.186667,0.042500
1,0.1,0.15,0.187083,0.045625
2,0.1,0.20,0.204583,0.049375
3,0.1,0.25,0.199583,0.050000
4,0.1,0.30,0.199583,0.050000
...,...,...,...,...
284,0.9,0.70,0.420000,0.050000
285,0.9,0.75,0.582500,0.050000
286,0.9,0.80,0.638750,0.050000
287,0.9,0.85,0.667083,0.050000
